In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("davidmckinley/all-the-news-dataset")

print("Path to dataset files:", path)

/Users/bing/.pyenv/versions/3.10.18/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/bing/.cache/kagglehub/datasets/davidmckinley/all-the-news-dataset/versions/1


In [3]:
import pandas as pd

df = pd.read_csv(path+"/all-the-news-2-1.csv", nrows=1)
print(df.columns)
print(df.shape)

"""
Index(['Unnamed: 0.1', 'Unnamed: 0', 'date', 'year', 'month', 'day', 'author',
       'title', 'article', 'url', 'section', 'publication'],
      dtype='object')
      """

Index(['Unnamed: 0.1', 'Unnamed: 0', 'date', 'year', 'month', 'day', 'author',
       'title', 'article', 'url', 'section', 'publication'],
      dtype='object')
(1, 12)


"\nIndex(['Unnamed: 0.1', 'Unnamed: 0', 'date', 'year', 'month', 'day', 'author',\n       'title', 'article', 'url', 'section', 'publication'],\n      dtype='object')\n      "

In [4]:
df = pd.read_csv(path+"/all-the-news-2-1.csv",nrows=1)
df['article'][0]

'This post is part of Polyarchy, an independent blog produced by the political reform program at New America, a Washington think tank devoted to developing new ideas and new voices. Imagine you are an otherwise healthy 30-something who starts feeling weird. You are sometimes short of breath. You get migraines. Your feet start to swell a little. But otherwise, everything seems fine. You go to the doctor. The doctor runs some tests. She tells you,\xa0It\'s probably nothing, but these could be signs of a coming heart attack. You push for more certainty, but the doctor tells you she\'s not sure. The human body is a complex system. You\'re young and otherwise pretty healthy. There could be plenty of other explanations for what you\'re feeling. But it is a little worrying. So just to be on the safe side, maybe you should reduce the stress in your life and eat a healthier diet. What would you do? If you\'re a sensible person, you\'d probably err on the side of precaution. Sure, it might be no

In [4]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 4.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [29]:
import pandas as pd
import vertexai
from google.genai import types
from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from neo4j import GraphDatabase
from qdrant_client import QdrantClient
from qdrant_client.http import models as q_models
from qdrant_client.http.models import PointStruct
import spacy
import re
import uuid
from tqdm import tqdm
import time

# =================== Load Data =====================
df = pd.read_csv(path + "/all-the-news-2-1.csv", nrows=1_000_000)
print(df.columns)

# ================== Configuration ==================
PROJECT_ID = "bing-tan-sndbx-c"
LOCATION = "europe-west4"
vertexai.init(project=PROJECT_ID, location=LOCATION)

# ---------------- Local Qdrant -------------------
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "news"
qdrant = QdrantClient(url=QDRANT_URL)

existing_collections = [c.name for c in qdrant.get_collections().collections]
if COLLECTION_NAME not in existing_collections:
    qdrant.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=q_models.VectorParams(
            size=3072,
            distance=q_models.Distance.COSINE
        )
    )

# ---------------- Local Neo4j ---------------------
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "testpassword"
neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# # ---------------- NLP Model -----------------------
nlp = spacy.load("en_core_web_sm")  # upgrade to _md or _lg for better results
embedding_model = TextEmbeddingModel.from_pretrained("gemini-embedding-001")  # preload once
model = GenerativeModel(model_name="gemini-2.0-flash-001")

Index(['Unnamed: 0.1', 'Unnamed: 0', 'date', 'year', 'month', 'day', 'author',
       'title', 'article', 'url', 'section', 'publication'],
      dtype='object')


/Users/bing/.pyenv/versions/3.10.18/lib/python3.10/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1761132763.687266 5745341 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
/Users/bing/.pyenv/versions/3.10.18/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ===================================================
# --- Embedding Helpers ---
# ===================================================
def embed_text(text: str) -> list:
    """Generate a single embedding"""
    return embedding_model.get_embeddings([text])[0].values


def embed_texts_batch(texts: list, batch_size: int = 100) -> list:
    """Batch embeddings with retry + rate limiting"""
    embeddings = []
    for start in range(0, len(texts), batch_size):
        batch = texts[start:start + batch_size]
        try:
            batch_embeddings = embedding_model.get_embeddings(batch)
            embeddings.extend([emb.values for emb in batch_embeddings])
        except Exception as e:
            print(f"⚠️ Vertex embedding batch failed ({e}), retrying...")
            time.sleep(3)
            try:
                batch_embeddings = embedding_model.get_embeddings(batch)
                embeddings.extend([emb.values for emb in batch_embeddings])
            except Exception as e:
                print(f"❌ Vertex embedding retry failed ({e})")
                embeddings.extend([None] * len(batch))
    return embeddings


# ===================================================
# --- NLP: Entities + Relations ---
# ===================================================
# def extract_entities_and_relations(text: str):
#     """Simple NER + dependency-based relation extraction"""
#     doc = nlp(text)
#     entities = [{"name": ent.text, "type": ent.label_} for ent in doc.ents]

#     relations = []
#     for sent in doc.sents:
#         subj = [tok for tok in sent if tok.dep_ in ("nsubj", "nsubjpass")]
#         obj = [tok for tok in sent if tok.dep_ in ("dobj", "pobj")]
#         verb = [tok for tok in sent if tok.pos_ == "VERB"]
#         if subj and obj and verb:
#             relations.append({
#                 "subject": subj[0].text,
#                 "predicate": verb[0].lemma_,
#                 "object": obj[0].text,
#                 "metadata": {"confidence": 0.8}
#             })
#     return entities, relations

def extract_entities_and_relations(text: str):
    """
    Ultra-intelligent entity and relation extraction using an LLM.
    Returns:
      entities: [{"name": str, "type": str}]
      relations: [{"subject": str, "predicate": str, "object": str, "metadata": {...}}]
    """

    prompt = f"""
    You are an expert information extraction model.
    Analyze the text below and extract:
      1. All named entities (with clear, semantic types: PERSON, ORG, LOCATION, EVENT, PRODUCT, etc.)
      2. All semantic relations between entities in subject-predicate-object format.

    Return the result as strict JSON with this structure:
    {{
      "entities": [{{"name": "...", "type": "..."}}],
      "relations": [{{"subject": "...", "predicate": "...", "object": "..."}}]
    }}

    Text:
    \"\"\"{text}\"\"\"
    """
    response = model.generate_content(
        contents=prompt, 
        generation_config={"response_mime_type": "application/json", "temperature": 0}
        )
    
    try:
        import json
        data = json.loads(response.text)
        entities = data.get("entities", [])
        relations = data.get("relations", [])
    except Exception:
        entities, relations = [], []
    # print(entities, relations)
    return entities, relations


def safe_rel_type(predicate: str) -> str:
    """Sanitize relation type for Neo4j"""
    if not predicate:
        return "RELATED_TO"
    clean = re.sub(r'[^A-Za-z0-9_]', '_', predicate.upper())
    return clean or "RELATED_TO"


# ===================================================
# --- Neo4j Ingestion ---
# ===================================================
def ingest_to_kg(article_id: str, title: str, entities: list,
                 relations: list, metadata: dict):
    """Insert article nodes, entities, and relations into Neo4j"""

    def _execute(tx):
        tx.run("""
            MERGE (a:Article {id: $article_id})
            SET a.title = $title,
                a.date = $date,
                a.section = $section,
                a.publication = $publication,
                a.author = $author
        """, article_id=article_id, title=title, **metadata)

        # Entities
        for ent in entities:
            tx.run("""
                MERGE (e:Entity {name: $name, type: $type})
                MERGE (a:Article {id: $article_id})
                MERGE (a)-[:MENTIONS]->(e)
            """, name=ent["name"], type=ent["type"], article_id=article_id)

        # Relations
        for rel in relations:
            rel_type = safe_rel_type(rel.get("predicate"))
            tx.run(f"""
                MERGE (s:Entity {{name: $subject}})
                MERGE (o:Entity {{name: $object}})
                MERGE (s)-[r:{rel_type}]->(o)
                SET r += $metadata
            """, subject=rel["subject"], object=rel["object"],
                   metadata=rel.get("metadata", {}))

    try:
        with neo4j_driver.session() as session:
            session.execute_write(_execute)
    except Exception as e:
        print(f"⚠️ Neo4j insert failed for article {article_id}: {e}")


# ===================================================
# --- Qdrant Ingestion (chunked) ---
# ===================================================
def ingest_to_qdrant(article_id: str, title: str, chunks: list,
                     chunk_embeddings: list, entities: list, metadata: dict):
    """Insert chunked article segments into Qdrant"""
    points = []
    for i, (chunk_text, emb) in enumerate(zip(chunks, chunk_embeddings)):
        if emb is None:
            continue
        payload = {
            "article_id": article_id,
            "chunk_id": i,
            "title": title,
            "content": chunk_text,
            "entities": entities,
            **metadata
        }
        points.append(PointStruct(
            id=str(uuid.uuid4()),
            vector=emb,
            payload=payload
        ))

    try:
        if points:
            qdrant.upsert(collection_name=COLLECTION_NAME, points=points)
    except Exception as e:
        print(f"⚠️ Qdrant insert failed for {title}: {e}")


# ===================================================
# --- Main DataFrame Ingestion ---
# ===================================================
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

def ingest_dataframe(df: pd.DataFrame, batch_size=50, embed_batch_size=100):
    """Ingest a dataframe into Qdrant + Neo4j with chunking"""
    for start in tqdm(range(0, len(df), batch_size)):
        batch = df.iloc[start:start + batch_size]

        for _, row in batch.iterrows():
            try:
                article_id = str(row.get("Unnamed: 0", uuid.uuid4()))
                title = row.get("title", "Untitled")
                content = row.get("article", "")
                metadata = {
                    "author": row.get("author"),
                    "date": row.get("date"),
                    "section": row.get("section"),
                    "publication": row.get("publication")
                }

                # --- Split article into chunks ---
                chunks = splitter.split_text(content)

                # --- Embed all chunks ---
                chunk_embeddings = embed_texts_batch(chunks, batch_size=embed_batch_size)

                # --- Extract entities & relations (once per article) ---
                entities, relations = extract_entities_and_relations(content)

                # --- Ingest chunked embeddings into Qdrant ---
                ingest_to_qdrant(article_id, title, chunks, chunk_embeddings, entities, metadata)

                # --- Ingest entities + relations into Neo4j ---
                ingest_to_kg(article_id, title, entities, relations, metadata)

                print(f"✅ Ingested article '{title}' ({len(chunks)} chunks)")
            except Exception as e:
                print(f"⚠️ Error ingesting article '{title}': {e}")


# ===================================================
# --- Script Entry Point ---
# ===================================================
if __name__ == "__main__":
    # Example: only first 100 rows for testing
    ingest_dataframe(df.sample(1000))


  0%|          | 0/20 [00:00<?, ?it/s]

⚠️ Vertex embedding batch failed (503 recvmsg:Connection reset by peer), retrying...
⚠️ Error ingesting article 'Texas' Brazos River hits century high, Houston braces for floods': 503 recvmsg:Connection reset by peer
✅ Ingested article 'Real Housewives of Beverly Hills' Eden Sassoon Talks Kim & Kyle Richards Drama' (4 chunks)
✅ Ingested article 'Federal investigators want to know if Facebook lied about Cambridge Analytica' (5 chunks)
✅ Ingested article 'Coalition strikes roads to prevent ISIS convoy from evacuating border area with Lebanon' (8 chunks)
✅ Ingested article 'Dozens of sharks in intense feeding frenzy on whale in Australia' (3 chunks)
✅ Ingested article 'Chris Tucker Steps Out of Comfort Zone in New Billy Lynn's Long Halftime Walk Role' (1 chunks)
✅ Ingested article 'Donald Trump desperately needs interpreters for his 'chat' with Kim' (18 chunks)
✅ Ingested article 'Wendy Williams' Estranged Husband's No Closer to Reconciling with Son' (4 chunks)
✅ Ingested article 'BRIEF-N

  5%|▌         | 1/20 [05:08<1:37:36, 308.23s/it]

✅ Ingested article 'Puerto Rico Blond Hair Unity - World Baseball Classic' (7 chunks)
✅ Ingested article 'People Who Own Cryptocurrency Are Getting Slammed With More Taxes Than They Expected' (16 chunks)
✅ Ingested article 'Wearable Technology Mostly Sucks. Here's What Needs to Change.' (6 chunks)
✅ Ingested article 'Kate Upton and Justin Verlander Are Getting Married This Weekend' (7 chunks)
✅ Ingested article 'In setback for 5-Star, Italy's PM backs completion of rail link with France' (8 chunks)
✅ Ingested article 'Facebook will let any WordPress blog post Instant Articles' (4 chunks)
✅ Ingested article 'Justin Bieber Stays in the Same $10,000 a Night Airbnb as Beyonce' (1 chunks)
✅ Ingested article 'Prince: 911 Caller Tight-lipped During Death Investigation' (3 chunks)
✅ Ingested article 'We Need to Stop Treating Naturopaths Like They’re Medical Doctors' (13 chunks)
✅ Ingested article 'Senior military official says snipers shooting civilians and soldiers in Khartoum' (3 chunks)
✅ I

 10%|█         | 2/20 [09:44<1:26:50, 289.45s/it]

...

✅ Ingested article 'Daphne Oz Embraces Cheat Days Into Her Post-Baby Meal Plan' (6 chunks)
✅ Ingested article 'Amanda Bynes Twitter Return' (3 chunks)
✅ Ingested article 'NFL, NFLPA Determine Dolphins Did Not Follow Concussion Protocol When Matt Moore Almost Got Decapitated' (8 chunks)
✅ Ingested article 'BRIEF-Petrowest Corporation announces senior management changes' (1 chunks)
✅ Ingested article 'Someone's Making an EDM Musical About American Expats in Berlin' (5 chunks)
✅ Ingested article 'Girlboss Show Vs Book Differences Sophia Amoruso Life' (4 chunks)
✅ Ingested article 'Cletus the Special Needs Dog Finds Home Through Best Friends' (5 chunks)
✅ Ingested article 'Jennifer Aniston Self Doubt Inspiring Quotes Giffoni' (7 chunks)
✅ Ingested article 'Sri Lanka deploys police, military after Buddhist-Muslim clash' (11 chunks)
✅ Ingested article 'Scoot wants to expand its electric scooter operations globally with a white-label service – TechCrunch' (7 chunks)
✅ Ingested article 'The li

 20%|██        | 4/20 [35:56<3:02:43, 685.22s/it]

✅ Ingested article 'BRIEF-RBC Global Asset Management reports $819 mln in mutual fund net sales for May' (2 chunks)
✅ Ingested article 'Fingerlings Glitter Monkeys on Sale at Amazon' (9 chunks)
✅ Ingested article 'Renault will vote to back Nissan's move to give it boardroom seats' (3 chunks)
✅ Ingested article 'Global money-laundering watchdog launches crackdown on cryptocurrencies' (12 chunks)
✅ Ingested article 'Liberal Lansing is divided over whether to become a sanctuary city â€“ VICE News' (3 chunks)
✅ Ingested article 'Trump agrees to keep US troops in Syria to defeat ISIS' (17 chunks)
✅ Ingested article 'Hitting a Low to Come Up Fearless: Allison Crutchfield’s Next Chapter ' (21 chunks)
✅ Ingested article 'New report details Trump campaign cash' (5 chunks)
✅ Ingested article 'North and South Korea start to dismantle border speakers, fulfilling summit pledge' (10 chunks)
✅ Ingested article 'UPDATE 2-New Pakistani government seeks to renegotiate LNG terminal deals - minister' (9 c

 20%|██        | 4/20 [1:05:28<4:21:54, 982.16s/it]


KeyboardInterrupt: 

In [41]:
from collections import Counter

# Get all pairs
pairs = [
    (p.payload.get("article_id"), p.payload.get("chunk_id"))
    for p in points
    if "article_id" in p.payload and "chunk_id" in p.payload
]

# Count occurrences
pair_counts = Counter(pairs)

# Extract duplicates
dupes = [pair for pair, count in pair_counts.items() if count > 1]

print(f"Total pairs: {len(pairs)}")
print(f"Unique pairs: {len(pair_counts)}")
print(f"Duplicate pairs: {len(dupes)}")

# Optional: show some examples
for pair in dupes[:5]:
    print(pair, "→", pair_counts[pair], "occurrences")


Total pairs: 4231
Unique pairs: 4231
Duplicate pairs: 0


In [38]:
points

[Record(id='0016fb85-cdf0-4609-ba9e-80b64e548c3f', payload={'article_id': '436652', 'chunk_id': 4, 'title': "Mozilla co-founder's Brave files adtech complaint against Google", 'content': 'against unauthorized or unlawful processing and against accidental loss. Google says it is has already implemented strong privacy protections in consultation with European regulators and is committed to complying with the GDPR. Brave operates as a private browser and ad blocker, preventing the use of trackers on web pages to harvest data about people’s online behavior - giving it detailed insight', 'entities': [{'name': 'COLOGNE', 'type': 'LOCATION'}, {'name': 'Germany', 'type': 'LOCATION'}, {'name': 'Reuters', 'type': 'ORG'}, {'name': 'Brave', 'type': 'ORG'}, {'name': 'Silicon Valley', 'type': 'LOCATION'}, {'name': 'Brendan Eich', 'type': 'PERSON'}, {'name': 'Britain', 'type': 'LOCATION'}, {'name': 'Ireland', 'type': 'LOCATION'}, {'name': 'Google', 'type': 'ORG'}, {'name': 'European General Data Prot

In [36]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

vectors = np.array([p.vector for p in points])
sims = cosine_similarity(vectors)
dupe_pairs = np.argwhere(sims > 0.9999)

print(f"Found {len(dupe_pairs)} nearly identical vectors.")


ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan ... nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [37]:
vectors

array([None, None, None, ..., None, None, None], dtype=object)

In [5]:
# Fetch all articles
with neo4j_driver.session() as session:
    articles = session.run("MATCH (a:Article) RETURN a.id AS id, a.title AS title")
    print("Articles in KG:")
    for a in articles:
        print(a["id"], "-", a["title"])

# Fetch all entities
with neo4j_driver.session() as session:
    entities = session.run("MATCH (e:Entity) RETURN e.name AS name, e.type AS type")
    print("\nEntities in KG:")
    for e in entities:
        print(e["name"], "-", e["type"])

# Fetch all relationships
with neo4j_driver.session() as session:
    rels = session.run("""
        MATCH (s:Entity)-[r]->(o:Entity)
        RETURN s.name AS subject, type(r) AS predicate, o.name AS object, r
    """)
    print("\nRelationships in KG:")
    for r in rels:
        print(f"{r['subject']} -[{r['predicate']}]-> {r['object']}")


Articles in KG:
['0801ae05-56c8-4503-8078-13532f08a2a2'] - The Second Coming of Betty Tompkins
['2f0d3511-437d-4c37-bc45-9c6d6f3f20c3'] - NFL playoffs: How to live stream the championship games today 
['9943258b-037a-41e4-96f3-ed6d43e25e48'] - Iran rejects U.S. warning against space launches, ballistic missiles
['cfa73e88-f388-4e21-b851-ebff1b3d0b5f'] - Philippines' Globe Telecoms launches 5G service backed by Huawei equipment
['6a5d0f30-bde7-4ccf-8b85-c4e7f4003f59'] - Oil-Rich Oklahoma Has Been  Hit by Nearly Three Dozen Earthquakes Since Wednesday
['3876bc67-24f7-4a36-a0c5-22544bc564c5'] - Trump tells anti-abortion marchers he will support them
['c674e99c-3e10-4a88-9269-4ce46fbbd59a'] - Despite climate pledges, G20 coal subsidies rise
['107136cf-7e29-4ca9-9c88-f497c2883324'] - Adam Rippon Says Aly Raisman Is Changing Dangerous Sports Culture
['6bb5911c-34d5-4e08-b1f4-5735ab813766'] - Why Jews Are Getting Themselves Arrested at ICE Centers Around the Country
['3698cf4d-d058-43dc-aea8-

In [6]:
with neo4j_driver.session() as session:
    result = session.run("""
        MATCH (a:Article)-[:MENTIONS]->(e:Entity)
        RETURN a.title AS title, COUNT(e) AS entity_count
    """)
    print("Entity mentions per article:")
    for r in result:
        print(f"{r['title']}: {r['entity_count']} entities")


Entity mentions per article:
The Second Coming of Betty Tompkins: 73 entities
NFL playoffs: How to live stream the championship games today : 25 entities
Iran rejects U.S. warning against space launches, ballistic missiles: 18 entities
Philippines' Globe Telecoms launches 5G service backed by Huawei equipment: 34 entities
Oil-Rich Oklahoma Has Been  Hit by Nearly Three Dozen Earthquakes Since Wednesday: 61 entities
Trump tells anti-abortion marchers he will support them: 50 entities
Despite climate pledges, G20 coal subsidies rise: 52 entities
Adam Rippon Says Aly Raisman Is Changing Dangerous Sports Culture: 9 entities
Why Jews Are Getting Themselves Arrested at ICE Centers Around the Country: 117 entities
The Sushi Chef: Shinichi Inoue: 8 entities
Why Obama is vetting Nevada's Republican governor for the Supreme Court: 47 entities
Exclusive: Trump considering fracking mogul Harold Hamm as energy secretary - sources: 57 entities
Photos: These are the Rohingya children who escaped Myan

In [ ]:
# import pandas as pd
# import vertexai
# from vertexai.preview.language_models import TextEmbeddingModel
# from neo4j import GraphDatabase
# from qdrant_client import QdrantClient
# from qdrant_client.http import models as q_models
# from qdrant_client.http.models import PointStruct
# import spacy
# import uuid
# from tqdm import tqdm

# # =================== Load Data =====================
# df = pd.read_csv(path + "/all-the-news-2-1.csv", nrows=10000)
# print(df.columns)

# # ================== Configuration ==================
# PROJECT_ID = "bing-tan-sndbx-c"
# LOCATION = "europe-west4"
# vertexai.init(project=PROJECT_ID, location=LOCATION)

# # ---------------- Local Qdrant -------------------
# QDRANT_URL = "http://localhost:6333"
# COLLECTION_NAME = "news"
# qdrant = QdrantClient(url=QDRANT_URL)

# existing_collections = [c.name for c in qdrant.get_collections().collections]
# if COLLECTION_NAME not in existing_collections:
#     qdrant.recreate_collection(
#         collection_name=COLLECTION_NAME,
#         vectors_config=q_models.VectorParams(
#             size=3072,
#             distance=q_models.Distance.COSINE
#         )
#     )

# # ---------------- Local Neo4j ---------------------
# NEO4J_URI = "bolt://localhost:7687"
# NEO4J_USER = "neo4j"
# NEO4J_PASSWORD = "testpassword"
# neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# # ---------------- NLP Model -----------------------
# nlp = spacy.load("en_core_web_sm")  # or a bigger NER model if available

# # ================= Helper Functions =================

# def embed_text(text: str) -> list:
#     """Generate embedding using Vertex AI"""
#     model = TextEmbeddingModel.from_pretrained("gemini-embedding-001")
#     embedding = model.get_embeddings([text])[0].values
#     return embedding

# def extract_entities_and_relations(text: str):
#     """NER + simple relation extraction using spaCy"""
#     doc = nlp(text)
#     entities = [{"name": ent.text, "type": ent.label_} for ent in doc.ents]

#     # Very simple heuristic relation extraction: Subject-Verb-Object patterns
#     relations = []
#     for sent in doc.sents:
#         subj = [tok for tok in sent if tok.dep_ in ("nsubj", "nsubjpass")]
#         obj = [tok for tok in sent if tok.dep_ in ("dobj", "pobj")]
#         verb = [tok for tok in sent if tok.pos_ == "VERB"]
#         if subj and obj and verb:
#             relations.append({
#                 "subject": subj[0].text,
#                 "predicate": verb[0].lemma_.upper().replace(" ", "_"),
#                 "object": obj[0].text,
#                 "metadata": {"confidence": 0.8}
#             })
#     return entities, relations

# def ingest_to_kg_and_vector(article_id: str, title: str, content: str, entities: list, relations: list, embedding: list, metadata: dict):
#     """Ingest one article into local Qdrant and Neo4j"""
#     # --- Qdrant ---
#     qdrant.upsert(
#         collection_name=COLLECTION_NAME,
#         points=[
#             PointStruct(
#                 id=str(uuid.uuid4()),
#                 vector=embedding,
#                 payload={
#                     "article_id": article_id,
#                     "title": title,
#                     "entities": entities,
#                     **metadata
#                 },
#             )
#         ],
#     )

#     # --- Neo4j ---
#     def _execute(tx):
#         tx.run(
#             """
#             MERGE (a:Article {id: $article_id})
#             SET a.title = $title, a.content = $content
#             """,
#             article_id=article_id, title=title, content=content
#         )
#         for ent in entities:
#             tx.run(
#                 """
#                 MERGE (e:Entity {name: $name, type: $type})
#                 MERGE (a:Article {id: $article_id})
#                 MERGE (a)-[:MENTIONS]->(e)
#                 """,
#                 name=ent["name"], type=ent["type"], article_id=article_id
#             )
#         for rel in relations:
#             tx.run(
#                 f"""
#                 MERGE (s:Entity {{name: $subject}})
#                 MERGE (o:Entity {{name: $object}})
#                 MERGE (s)-[r:{rel['predicate']}]->(o)
#                 SET r += $metadata
#                 """,
#                 subject=rel["subject"], object=rel["object"], metadata=rel.get("metadata", {})
#             )

#     with neo4j_driver.session() as session:
#         session.execute_write(_execute)

# def ingest_article(article_json: dict):
#     """Ingest a single article"""
#     article_id = article_json.get("id", str(uuid.uuid4()))
#     title = article_json["title"]
#     content = article_json["content"]
#     metadata = {k: article_json.get(k) for k in ["author", "date", "section", "publication"]}

#     embedding = embed_text(content)
#     entities, relations = extract_entities_and_relations(content)
#     ingest_to_kg_and_vector(article_id, title, content, entities, relations, embedding, metadata)

# def embed_texts_batch(texts: list, batch_size: int = 1000) -> list:
#     """
#     Generate embeddings for a batch of texts using Vertex AI.
#     Returns a list of embeddings in the same order as input texts.
#     """
#     model = TextEmbeddingModel.from_pretrained("gemini-embedding-001")
#     embeddings = []
#     for start in range(0, len(texts), batch_size):
#         batch = texts[start:start+batch_size]
#         batch_embeddings = model.get_embeddings(batch)
#         embeddings.extend([emb.values for emb in batch_embeddings])
#     return embeddings

# # ================= Batch Ingestion =================

# def ingest_dataframe(df: pd.DataFrame, chunk_size=50, embed_batch_size=1000):
#     for start in tqdm(range(0, len(df), chunk_size)):
#         chunk = df.iloc[start:start+chunk_size]
        
#         # Prepare data for embedding
#         texts = chunk["article"].tolist()
#         embeddings = embed_texts_batch(texts, batch_size=embed_batch_size)
        
#         for i, (_, row) in enumerate(chunk.iterrows()):
#             article_json = {
#                 "id": str(row.get("Unnamed: 0", uuid.uuid4())),
#                 "title": row["title"],
#                 "content": row["article"],
#                 "author": row.get("author"),
#                 "date": row.get("date"),
#                 "section": row.get("section"),
#                 "publication": row.get("publication")
#             }
#             try:
#                 embedding = embeddings[i]  # Use precomputed embedding
#                 entities, relations = extract_entities_and_relations(article_json["content"])
#                 metadata = {k: article_json.get(k) for k in ["author", "date", "section", "publication"]}
#                 ingest_to_kg_and_vector(article_json["id"], article_json["title"], article_json["content"],
#                                         entities, relations, embedding, metadata)
#             except Exception as e:
#                 print(f"Error ingesting article {row.get('title')}: {e}")




# if __name__ == "__main__":
#     ingest_dataframe(df.head(100000))  # test with first 10 rows


Index(['Unnamed: 0.1', 'Unnamed: 0', 'date', 'year', 'month', 'day', 'author',
       'title', 'article', 'url', 'section', 'publication'],
      dtype='object')


/Users/bing/.pyenv/versions/3.10.18/lib/python3.10/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1760625864.617532 3517781 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
  0%|          | 0/2000 [00:00<?, ?it/s]E0000 00:00:1760625864.770577 3517781 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760625864.771287 3517781 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
  0%|          | 4/2000 [01:20<11:01:30, 19.89s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x110360220>>
T

In [25]:
def reset_neo4j():
    try:
        with neo4j_driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
        print("✅ Neo4j database reset successfully.")
    except Exception as e:
        print(f"⚠️ Failed to reset Neo4j: {e}")

reset_neo4j()


COLLECTION_NAME = "news"

# Delete the collection if it exists
if COLLECTION_NAME in [c.name for c in qdrant.get_collections().collections]:
    qdrant.delete_collection(COLLECTION_NAME)
    print(f"✅ Qdrant collection '{COLLECTION_NAME}' deleted.")

# Recreate empty collection
qdrant.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=q_models.VectorParams(
        size=3072,      # adjust to your embedding size
        distance=q_models.Distance.COSINE
    )
)
print(f"✅ Qdrant collection '{COLLECTION_NAME}' recreated.")


✅ Neo4j database reset successfully.
✅ Qdrant collection 'news' deleted.
✅ Qdrant collection 'news' recreated.


/var/folders/h1/d93890857d789cwc1wqytm440000gp/T/ipykernel_1218/584075327.py:20: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


In [ ]:


# # ---------------- NLP Model -----------------------
# nlp = spacy.load("en_core_web_sm")  # upgrade to _md or _lg for better results
# embedding_model = TextEmbeddingModel.from_pretrained("gemini-embedding-001")  # preload once

# # ================= Helper Functions =================

# def embed_text(text: str) -> list:
#     """Generate a single embedding"""
#     return embedding_model.get_embeddings([text])[0].values


# def embed_texts_batch(texts: list, batch_size: int = 100) -> list:
#     """Batch embeddings with retry + rate limiting"""
#     embeddings = []
#     for start in range(0, len(texts), batch_size):
#         batch = texts[start:start + batch_size]
#         try:
#             batch_embeddings = embedding_model.get_embeddings(batch)
#             embeddings.extend([emb.values for emb in batch_embeddings])
#         except Exception as e:
#             print(f"⚠️ Vertex embedding batch failed ({e}), retrying...")
#             time.sleep(3)
#             try:
#                 batch_embeddings = embedding_model.get_embeddings(batch)
#                 embeddings.extend([emb.values for emb in batch_embeddings])
#             except Exception as e:
#                 print(f"❌ Vertex embedding retry failed ({e})")
#                 # fill with None to keep indexing safe
#                 embeddings.extend([None] * len(batch))
#     return embeddings



# def extract_entities_and_relations(text: str):
#     """Simple NER + dependency-based relation extraction"""
#     doc = nlp(text)
#     entities = [{"name": ent.text, "type": ent.label_} for ent in doc.ents]

#     relations = []
#     for sent in doc.sents:
#         subj = [tok for tok in sent if tok.dep_ in ("nsubj", "nsubjpass")]
#         obj = [tok for tok in sent if tok.dep_ in ("dobj", "pobj")]
#         verb = [tok for tok in sent if tok.pos_ == "VERB"]
#         if subj and obj and verb:
#             relations.append({
#                 "subject": subj[0].text,
#                 "predicate": verb[0].lemma_,
#                 "object": obj[0].text,
#                 "metadata": {"confidence": 0.8}
#             })
#     return entities, relations


# def safe_rel_type(predicate: str) -> str:
#     """Sanitize relation type for Neo4j"""
#     if not predicate:
#         return "RELATED_TO"
#     clean = re.sub(r'[^A-Za-z0-9_]', '_', predicate.upper())
#     if not clean:
#         clean = "RELATED_TO"
#     return clean


# def ingest_to_kg_and_vector(article_id: str, title: str, content: str,
#                             entities: list, relations: list, embedding: list, metadata: dict):
#     """Ingest one article into Qdrant and Neo4j"""

#     # --- Qdrant ---
#     try:
#         qdrant.upsert(
#             collection_name=COLLECTION_NAME,
#             points=[
#                 PointStruct(
#                     id=str(uuid.uuid4()),
#                     vector=embedding,
#                     payload={
#                         "article_id": article_id,
#                         "title": title,
#                         "content": content,
#                         "entities": entities,
#                         **metadata
#                     },
#                 )
#             ],
#         )
#     except Exception as e:
#         print(f"⚠️ Qdrant insert failed for article {article_id}: {e}")

#     # --- Neo4j ---
#     def _execute(tx):
#         tx.run("""
#             MERGE (a:Article {id: $article_id})
#             SET a.title = $title, a.content = $content
#             """, article_id=article_id, title=title, content=content)

#         for ent in entities:
#             tx.run("""
#                 MERGE (e:Entity {name: $name, type: $type})
#                 MERGE (a:Article {id: $article_id})
#                 MERGE (a)-[:MENTIONS]->(e)
#                 """, name=ent["name"], type=ent["type"], article_id=article_id)

#         for rel in relations:
#             rel_type = safe_rel_type(rel.get("predicate"))
#             tx.run(f"""
#                 MERGE (s:Entity {{name: $subject}})
#                 MERGE (o:Entity {{name: $object}})
#                 MERGE (s)-[r:{rel_type}]->(o)
#                 SET r += $metadata
#                 """,
#                 subject=rel["subject"], object=rel["object"], metadata=rel.get("metadata", {}))

#     try:
#         with neo4j_driver.session() as session:
#             session.execute_write(_execute)
#     except Exception as e:
#         print(f"⚠️ Neo4j insert failed for article {article_id}: {e}")


# def ingest_dataframe(df: pd.DataFrame, chunk_size=50, embed_batch_size=100):
#     """Ingest dataframe to Qdrant + Neo4j with batching and logging"""
#     for start in tqdm(range(0, len(df), chunk_size)):
#         chunk = df.iloc[start:start + chunk_size]
#         texts = chunk["article"].tolist()
#         embeddings = embed_texts_batch(texts, batch_size=embed_batch_size)

#         for i, (_, row) in enumerate(chunk.iterrows()):
#             article_json = {
#                 "id": str(uuid.uuid4()), #str(row.get("Unnamed: 0", uuid.uuid4())),
#                 "title": row["title"],
#                 "content": row["article"],
#                 "author": row.get("author"),
#                 "date": row.get("date"),
#                 "section": row.get("section"),
#                 "publication": row.get("publication")
#             }
#             try:
#                 entities, relations = extract_entities_and_relations(article_json["content"])
#                 metadata = {k: article_json.get(k) for k in ["author", "date", "section", "publication"]}
#                 ingest_to_kg_and_vector(
#                     article_json["id"], article_json["title"], article_json["content"],
#                     entities, relations, embeddings[i], metadata
#                 )
#             except Exception as e:
#                 print(f"⚠️ Error ingesting article {article_json['title']}: {e}")


# if __name__ == "__main__":
#     ingest_dataframe(df)  # test safely on first 100 rows
